# Question 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Nice Progress bar for loops
from Rankine_solver import define_empty_variables, solve_r_rankine_cycle
from Rankine_solver import display_tables, plot_Ts_cycle

In [ ]:
def Conductance(k,L,r1,r2):
    G=(2*np.pi*k*L)/(np.log(r2/r1))
    return G

In [ ]:
G=Conductance(398, 1, 0.003, 0.005)
print(G)

Starting w the cold side i.e. 1->2 

In [ ]:
#finding value of qc
vars = define_empty_variables()

# give the solver *one* piece of information per state
vars['2']['T'] = 5  # C
vars['4']['T'] = 35 # C

solved = solve_r_rankine_cycle(vars)

display_tables(solved)

In [ ]:
qc=151 #kJ/kg

We know $\dot{Q}_C=G\cdot \Delta T$ and $\dot{Q}_C=\dot{m} \cdot q_C$, therefore the maximum heat flow $\dot{m}_C^{Max}$ for the process $1\rightarrow{2}$ is :  $\dot{m}_C^{Max}=\frac{G\cdot \Delta T}{q_C}$

In [ ]:
Tc =10+273.15 #K
T2=5+273.15 #K
m_dot_C=(G*10**(-3)*(Tc-T2))/qc #kg/s
print(m_dot_C) 

Repeat for process $3 \rightarrow{4}$

In [ ]:
qh=171 #kJ/kg
TH=22+273.15 #K
T4=35+273.15 #K
m_dot_H=(G*10**(-3)*(np.abs(TH-T4)))/qh #kg/s
print(m_dot_H)


In [ ]:
#finding min value and assigning to m_dot 
m_dot=min(m_dot_C,m_dot_H)
print(m_dot)

In [ ]:
def get_q_from_T(T2, T4):
    vars = define_empty_variables()
    vars['2']['T'] = T2 # C
    vars['4']['T'] = T4 # C

    solved = solve_r_rankine_cycle(vars)
    return solved['qh'], solved['qc']



def get_m_dot_from_T(G,TC,TH,T2, T4):
    if T2 >= TC :
        return 0  # No heat absorption or rejection possible
    qh,qc=get_q_from_T(T2, T4)
    m_dot_H = 0
    if abs(qh) > 1e-5 :
        m_dot_H = (G * 1e-3 * np.abs(TH - T4)) / qh

    m_dot_C = 0
    if abs(qc) > 1e-5 :
        m_dot_C = (G * 1e-3 * np.abs(TC - T2)) / qc

    m_dot = max(min(m_dot_C, m_dot_H), 0)
    return m_dot


T2s = np.linspace(-10, 15, 100)  # C
T4s = [25, 30, 35]  # C
TC=10  # C
TH=22  # C
G=4895.423478997306 # W/K

plt.figure(figsize=(10, 6))
for T4 in T4s:
    print(f'Calculating Q_H for T4 = {T4}°C')
    Q_Hs = []
    for T2 in tqdm(T2s):
        Q_H = get_q_from_T(T2, T4)[0] * get_m_dot_from_T(G, TC, TH, T2, T4)
        Q_Hs.append(Q_H)
    plt.plot(T2s, Q_Hs, label=f'T4 = {T4}°C')

plt.xlabel(r'$T_2$ (°C)')
plt.ylabel(r'$\dot{Q}_H$ (kW)')
plt.title('Maximum heating power for different T4 values')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
qh=get_q_from_T(0, 35)[0]
m_dot=get_m_dot_from_T(G,10,22,0, 35)
QH1=qh*m_dot
print(QH1)